In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import glob
import json

import pandas as pd

In [18]:
import os

big_data = pd.DataFrame()
top_level_folder_list = [
    'configs/harmony/recursive/harmony_recurv_auenabled_longonly_1.0WE_bwtp_off',
    'configs/harmony/recursive/harmony_recurv_auenabled_longshort_0.25_0.12WE_bwtp_off',
    'configs/harmony/recursive/harmony_recurv_noau_longshort_0.25_0.12WE_bwtpoff',
    'configs/harmony/recursive/harmony_recurv_noau_shortonly_1.0WE_bwtp_off',
    'configs/particle_swarm/neat/neat_grid_nov_20',
    'configs/particle_swarm/neat/PSO_scalper_40%mingrid',
    'configs/particle_swarm/neat/tight_0.3_0.4_long',
    'configs/particle_swarm/neat/wider_0.3_0.4_long',
    'configs/particle_swarm/recursive/November',
    'configs/particle_swarm/recursive/pso_recursive_auenabled_longshort',   
    'downloaded_configs/passivbot_v5.8.0/configs/harmony/recursive/harmony_recurv_auenabled_longonly_1.0WE_bwtp_off',
    'downloaded_configs/passivbot_v5.8.0/configs/particle_swarm/neat',
    'downloaded_configs/passivbot_v5.8.0/configs/particle_swarm/recursive/pso_recursive_auenabled_longshort',
    'downloaded_configs/passivbot_v5.8.0/configs/particle_swarm/recursive/pso_recursive_noau_longshort_0.1',
    'downloaded_configs/passivbot_v5.8.0/configs/particle_swarm/recursive/pso_recursive_noau_longshort_0.1_bwtp_off',
    'downloaded_configs/passivbot_v5.8.0/configs/particle_swarm/recursive/pso_recurv_noau_longonly_1.0WE',
    'downloaded_configs/passivbot_v5.8.0/configs/particle_swarm/recursive/pso_recurv_noau_longonly_1.0WE_BWTP_OFF',   
]
files = []
for folder in top_level_folder_list:
    for file in glob.iglob(folder + '/**/result.txt', recursive=True):
        files.append(file)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = os.path.abspath(filename.replace('result.txt', '../config.json'))
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())

        big_data = big_data.append(d, ignore_index=True)
big_data

C:\Users\skhan\AppData\Local\Temp\ipykernel_18820\3531623287.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_18820\3531623287.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_18820\3531623287.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_18820\3531623287.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_inde

,file,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,adgs_%,bnkrs_%,ebrs,maxss,pamaxs,pastds,short,tgs_%
0,downloaded_configs/passivbot_v5.8.0/configs/pa...,ATOMUSDT,True,18.290,0.0160,0.013326,0.565401,100.00,0.9642,3.568054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,downloaded_configs/passivbot_v5.8.0/configs/pa...,AVAXUSDT,True,12.140,0.0137,0.018205,0.413223,100.00,0.9710,9.336125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,downloaded_configs/passivbot_v5.8.0/configs/pa...,CHZUSDT,True,16.840,0.0227,0.010568,0.972791,100.00,0.9507,3.097221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,downloaded_configs/passivbot_v5.8.0/configs/pa...,DOGEUSDT,True,13.740,0.0144,0.009992,0.610235,100.00,0.9629,10.778458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,downloaded_configs/passivbot_v5.8.0/configs/pa...,DYDXUSDT,True,7.566,0.0163,0.009664,0.218598,100.00,0.9820,1.322917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,downloaded_configs/passivbot_v5.8.0/configs/pa...,HBARUSDT,True,209.500,0.1860,0.011753,0.498154,99.83,0.6671,9.752083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,downloaded_configs/passivbot_v5.8.0/configs/pa...,MATICUSDT,True,580.200,0.2610,0.012178,0.299799,99.85,0.7691,8.481958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,downloaded_configs/passivbot_v5.8.0/configs/pa...,SUSHIUSDT,True,523.000,0.2280,0.010983,0.919427,99.75,0.5204,2.986804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,downloaded_configs/passivbot_v5.8.0/configs/pa...,XRPUSDT,True,913.200,0.2230,0.009385,0.326524,99.86,0.7536,8.080542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
allowed_symbols = [
    "ETHUSDT", "BCHUSDT", "XRPUSDT", "EOSUSDT", "LTCUSDT", "TRXUSDT", "ETCUSDT", "LINKUSDT", "XLMUSDT", "ADAUSDT",
    "XMRUSDT", "DASHUSDT", "ZECUSDT", "XTZUSDT", "BNBUSDT", "ATOMUSDT", "NEOUSDT", "IOSTUSDT", "THETAUSDT", "ALGOUSDT",
    "ZILUSDT", "KNCUSDT", "COMPUSDT", "OMGUSDT", "DOGEUSDT", "KAVAUSDT", "BANDUSDT", "RLCUSDT", "WAVESUSDT", "SNXUSDT",
    "DOTUSDT", "CRVUSDT", "TRBUSDT", "RUNEUSDT", "SUSHIUSDT", "SRMUSDT", "EGLDUSDT", "FTMUSDT", "HNTUSDT", "RENUSDT",
    "KSMUSDT", "NEARUSDT", "FILUSDT", "RSRUSDT", "MATICUSDT", "OCEANUSDT", "CVCUSDT", "GRTUSDT", "1INCHUSDT", "CHZUSDT",
    "SANDUSDT", "BTSUSDT", "LITUSDT", "UNFIUSDT", "REEFUSDT", "MANAUSDT", "HBARUSDT", "ONEUSDT", "LINAUSDT", "STMXUSDT",
    "DENTUSDT", "HOTUSDT", "MTLUSDT", "SCUSDT", "1000SHIBUSDT"
]
#allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
    #& (big_data['long'] == True)
    #& (big_data['adgl_%'] > 0.04)
    #& (big_data['short'] == True)
    #& (big_data['adgt_%'] > 0.1)
    #& (big_data['bnkrl_%'] > 95.0)
    #& (big_data['bnkrs_%'] > 95.0)
    #& (big_data['ebrs'] > .85)
    #& (big_data['ebrl'] > .8)
    #& (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df.to_csv('5.8.csv')
df

,file,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,adgs_%,bnkrs_%,ebrs,maxss,pamaxs,pastds,short,tgs_%,adgt_%
0,downloaded_configs/passivbot_v5.8.0/configs/pa...,ATOMUSDT,True,18.290,0.01600,0.013326,0.565401,100.00,0.9642,3.568054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,downloaded_configs/passivbot_v5.8.0/configs/pa...,CHZUSDT,True,16.840,0.02270,0.010568,0.972791,100.00,0.9507,3.097221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,downloaded_configs/passivbot_v5.8.0/configs/pa...,DOGEUSDT,True,13.740,0.01440,0.009992,0.610235,100.00,0.9629,10.778458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,downloaded_configs/passivbot_v5.8.0/configs/pa...,EOSUSDT,True,7.393,0.00694,0.009094,0.419094,100.00,0.9704,3.384721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,downloaded_configs/passivbot_v5.8.0/configs/pa...,HBARUSDT,True,12.580,0.01890,0.010123,0.632434,100.00,0.9613,2.087500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,downloaded_configs/passivbot_v5.8.0/configs/pa...,HBARUSDT,True,209.500,0.18600,0.011753,0.498154,99.83,0.6671,9.752083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,downloaded_configs/passivbot_v5.8.0/configs/pa...,MATICUSDT,True,580.200,0.26100,0.012178,0.299799,99.85,0.7691,8.481958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,downloaded_configs/passivbot_v5.8.0/configs/pa...,SUSHIUSDT,True,523.000,0.22800,0.010983,0.919427,99.75,0.5204,2.986804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,downloaded_configs/passivbot_v5.8.0/configs/pa...,XRPUSDT,True,913.200,0.22300,0.009385,0.326524,99.86,0.7536,8.080542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.drop_duplicates(
    subset=['s',  'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = df[['file',  's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

C:\Users\skhan\AppData\Local\Temp\ipykernel_18820\994885060.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
0,downloaded_configs/passivbot_v5.8.0/configs/pa...,ATOMUSDT,NaN,0.01600,NaN,100.00,NaN,0.9642,NaN,3.568054,0.565401,0.013326,NaN,NaN,NaN
2,downloaded_configs/passivbot_v5.8.0/configs/pa...,CHZUSDT,NaN,0.02270,NaN,100.00,NaN,0.9507,NaN,3.097221,0.972791,0.010568,NaN,NaN,NaN
3,downloaded_configs/passivbot_v5.8.0/configs/pa...,DOGEUSDT,NaN,0.01440,NaN,100.00,NaN,0.9629,NaN,10.778458,0.610235,0.009992,NaN,NaN,NaN
5,downloaded_configs/passivbot_v5.8.0/configs/pa...,EOSUSDT,NaN,0.00694,NaN,100.00,NaN,0.9704,NaN,3.384721,0.419094,0.009094,NaN,NaN,NaN
7,downloaded_configs/passivbot_v5.8.0/configs/pa...,HBARUSDT,NaN,0.01890,NaN,100.00,NaN,0.9613,NaN,2.087500,0.632434,0.010123,NaN,NaN,NaN
8,downloaded_configs/passivbot_v5.8.0/configs/pa...,MATICUSDT,NaN,0.02970,NaN,100.00,NaN,0.9722,NaN,3.371529,0.382831,0.009755,NaN,NaN,NaN
9,downloaded_configs/passivbot_v5.8.0/configs/pa...,NEARUSDT,NaN,0.01590,NaN,100.00,NaN,0.9707,NaN,2.759721,0.416995,0.009777,NaN,NaN,NaN
11,downloaded_configs/passivbot_v5.8.0/configs/pa...,SANDUSDT,NaN,0.01610,NaN,100.00,NaN,0.9670,NaN,3.961113,0.490364,0.012469,NaN,NaN,NaN
13,downloaded_configs/passivbot_v5.8.0/configs/pa...,TRBUSDT,NaN,0.00627,NaN,100.00,NaN,0.9645,NaN,3.060417,0.550375,0.016056,NaN,NaN,NaN
14,downloaded_configs/passivbot_v5.8.0/configs/pa...,XTZUSDT,NaN,0.01360,NaN,100.00,NaN,0.9475,NaN,4.122917,1.109590,0.009704,NaN,NaN,NaN


In [25]:
dl_proposed = dx[
    #(dx['lm'] == 'n') &
    (dx['bnkrl_%'] == 100) &
    (dx['ebrl'] > 0.9) &  #&
    ((dx['ebrs'] > 0.9) | (dx['ebrs'].isnull())) &  #&
    # (dx['adgl_%'] > 0.04) 
    (dx['pastdl'] < 0.1) &
    ((dx['pastds'] < 0.1) | (dx['pastds'].isnull())) &  #
    (dx['maxsl'] < 10)  #&
    #  (dx['maxss'] < 10)
    ]
#dl_proposed = dl_proposed[['file','s','adgl_%', 'bnkrl_%', 'ebrl',  'maxsl', 'pamaxl','pastdl' ]]
dl_proposed

,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
0,downloaded_configs/passivbot_v5.8.0/configs/pa...,ATOMUSDT,NaN,0.01600,NaN,100.0,NaN,0.9642,NaN,3.568054,0.565401,0.013326,NaN,NaN,NaN
2,downloaded_configs/passivbot_v5.8.0/configs/pa...,CHZUSDT,NaN,0.02270,NaN,100.0,NaN,0.9507,NaN,3.097221,0.972791,0.010568,NaN,NaN,NaN
5,downloaded_configs/passivbot_v5.8.0/configs/pa...,EOSUSDT,NaN,0.00694,NaN,100.0,NaN,0.9704,NaN,3.384721,0.419094,0.009094,NaN,NaN,NaN
7,downloaded_configs/passivbot_v5.8.0/configs/pa...,HBARUSDT,NaN,0.01890,NaN,100.0,NaN,0.9613,NaN,2.087500,0.632434,0.010123,NaN,NaN,NaN
8,downloaded_configs/passivbot_v5.8.0/configs/pa...,MATICUSDT,NaN,0.02970,NaN,100.0,NaN,0.9722,NaN,3.371529,0.382831,0.009755,NaN,NaN,NaN
9,downloaded_configs/passivbot_v5.8.0/configs/pa...,NEARUSDT,NaN,0.01590,NaN,100.0,NaN,0.9707,NaN,2.759721,0.416995,0.009777,NaN,NaN,NaN
11,downloaded_configs/passivbot_v5.8.0/configs/pa...,SANDUSDT,NaN,0.01610,NaN,100.0,NaN,0.9670,NaN,3.961113,0.490364,0.012469,NaN,NaN,NaN
13,downloaded_configs/passivbot_v5.8.0/configs/pa...,TRBUSDT,NaN,0.00627,NaN,100.0,NaN,0.9645,NaN,3.060417,0.550375,0.016056,NaN,NaN,NaN
14,downloaded_configs/passivbot_v5.8.0/configs/pa...,XTZUSDT,NaN,0.01360,NaN,100.0,NaN,0.9475,NaN,4.122917,1.109590,0.009704,NaN,NaN,NaN
15,downloaded_configs/passivbot_v5.8.0/configs/pa...,ATOMUSDT,0.02580,0.01320,0.01260,100.0,100.0,0.9567,0.9769,7.978458,0.729742,0.026962,6.379875,0.208324,0.028154
